In [1]:
from pyspark.sql import SparkSession
import os

In [12]:
spark = SparkSession.builder \
.master("local[4]") \
.appName("TimeOps") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
.getOrCreate()

In [13]:
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [14]:
df = spark.read \
.option("header", "True") \
.option("inferSchema", "True") \
.option("sep", ";") \
.csv(f"{os.getcwd()}/OnlineRetail.csv")

In [15]:
df.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



In [16]:
df.show(15)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|1.12.2010 08:26|     7,65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|1.12.2010 08:

In [17]:
current_format = 'dd.MM.yyyy HH:mm'

In [18]:
from pyspark.sql import functions as F

df2 = df.withColumn("InvoiceDate", F.trim(F.col("InvoiceDate"))) \
.withColumn("normal_tarih", F.to_date(F.col("InvoiceDate"), current_format)) \
.withColumn("standart_ts", F.to_timestamp(F.col("InvoiceDate"), current_format)) \

df2.show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+------------+-------------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|normal_tarih|        standart_ts|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+------------+-------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|  2010-12-01|2010-12-01 08:26:00|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|  2010-12-01|2010-12-01 08:26:00|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|  2010-12-01|2010-12-01 08:26:00|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|  2010-12-01|2010-12-01 08:26:00|
|   536365|   84029E|RED WOOLLY HOTTIE...

In [19]:
format_tr = "dd/MM/yyyy HH:mm:ss"
format_eng = "MM-dd-yyyy HH:mm:ss"

df3 = df2 \
.withColumn("TSTR", F.date_format(F.col("standart_ts"), format_tr)) \
.withColumn("TSENG", F.date_format(F.col("standart_ts"), format_eng)) \
.withColumn("unix_time", F.unix_timestamp(F.col("standart_ts"))) \

df3.show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+------------+-------------------+-------------------+-------------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|normal_tarih|        standart_ts|               TSTR|              TSENG| unix_time|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+------------+-------------------+-------------------+-------------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|  2010-12-01|2010-12-01 08:26:00|01/12/2010 08:26:00|12-01-2010 08:26:00|1291184760|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|  2010-12-01|2010-12-01 08:26:00|01/12/2010 08:26:00|12-01-2010 08:26:00|1291184760|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010

In [20]:
df4 = df2 \
.withColumn("bir_yil", F.date_add(F.col("standart_ts"), 365)) \
.withColumn("yil", F.year(F.col("standart_ts"))) \
.withColumn("fark", F.datediff(F.col("bir_yil"), F.col("standart_ts")))

df4.show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+------------+-------------------+----------+----+----+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|normal_tarih|        standart_ts|   bir_yil| yil|fark|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+------------+-------------------+----------+----+----+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|  2010-12-01|2010-12-01 08:26:00|2011-12-01|2010| 365|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|  2010-12-01|2010-12-01 08:26:00|2011-12-01|2010| 365|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|  2010-12-01|2010-12-01 08:26:00|2011-12-01|2010| 365|
|   536365|   84029G|KNITTED UNION FLA...|       6|1